# 1. Importaciones y descargas

In [2]:
import re
from unidecode import unidecode
import stanza
from whoosh import index
from whoosh.fields import Schema, TEXT
from whoosh.qparser import QueryParser

# Descargar recursos de stanza
stanza.download('es',package='ancora', processors='tokenize,mwt,pos,lemma', verbose=True) 

# Inicializar el pipeline de stanza
stNLP = stanza.Pipeline(processors='tokenize,mwt,pos,lemma',lang='es',use_gpu=True)


2025-03-19 19:19:53 INFO: Downloaded file to C:\Users\alumnoBIGDATA\stanza_resources\resources.json
2025-03-19 19:19:53 INFO: Downloading these customized packages for language: es (Spanish)...
| Processor       | Package         |
-------------------------------------
| tokenize        | ancora          |
| mwt             | ancora          |
| pos             | ancora_charlm   |
| lemma           | ancora_nocharlm |
| pretrain        | conll17         |
| backward_charlm | newswiki        |
| forward_charlm  | newswiki        |

2025-03-19 19:19:53 INFO: File exists: C:\Users\alumnoBIGDATA\stanza_resources\es\tokenize\ancora.pt
2025-03-19 19:19:53 INFO: File exists: C:\Users\alumnoBIGDATA\stanza_resources\es\mwt\ancora.pt
2025-03-19 19:19:53 INFO: File exists: C:\Users\alumnoBIGDATA\stanza_resources\es\pos\ancora_charlm.pt
2025-03-19 19:19:53 INFO: File exists: C:\Users\alumnoBIGDATA\stanza_resources\es\lemma\ancora_nocharlm.pt
2025-03-19 19:19:53 INFO: File exists: C:\Users\alumnoBI

# 2. Leer fichero

In [3]:
def leer_fichero(ruta_fichero):
    with open(ruta_fichero, "r", encoding='utf-8') as f:
        contenido = f.read()
    return contenido

# 3. Extraer secciones, limpiar texto y extraer sustantivos

In [4]:
def extraer_secciones(contenido):
    secciones = {}
    # Usar expresiones regulares para encontrar las secciones
    titulo_match = re.search(r'Titulo(.*?)(?=Noticia|Resumen|$)', contenido, re.DOTALL)
    noticia_match = re.search(r'Noticia(.*?)(?=Resumen|$)', contenido, re.DOTALL)
    resumen_match = re.search(r'Resumen(.*?)(?=$)', contenido, re.DOTALL)

    if titulo_match:
        secciones['titulo'] = titulo_match.group(1).strip()
    if noticia_match:
        secciones['noticia'] = noticia_match.group(1).strip()
    if resumen_match:
        secciones['resumen'] = resumen_match.group(1).strip()

    return secciones

def limpiar_texto(texto):
    texto = re.sub(r'\s+', ' ', texto)  # Reemplazar múltiples espacios por uno solo
    texto = re.sub(r'[.,;!?]', '', texto)  # Eliminar puntuación
    return texto

def extraer_sustantivos(texto):
    doc = stNLP(texto)
    sustantivos = set()  # Usamos set para evitar duplicados
    
    for sent in doc.sentences:
        for word in sent.words:
            if word.pos == 'NOUN' or word.pos == 'PROPN':
                sustantivos.add(word.text)

    return sustantivos

# 6. Buscador woosh

In [18]:
import os

# Definir el esquema de Whoosh
esquema = Schema(seccion=TEXT(stored=True), sustantivos=TEXT(stored=True))

def crear_indice(sustantivos):
    if not os.path.exists("indice_sustantivos"):
        os.mkdir("indice_sustantivos")
    
    index.create_in("indice_sustantivos", esquema)
    indice = index.open_dir("indice_sustantivos")
    writer = indice.writer()
    
    for seccion, palabras in sustantivos.items():
        writer.add_document(seccion=seccion, sustantivos=" ".join(palabras))
    
    writer.commit()

def buscar_sustantivo(seccion, sustantivo):
    indice = index.open_dir("indice_sustantivos")
    parser = QueryParser("sustantivos", schema=esquema)
    consulta = parser.parse(sustantivo)
    
    with indice.searcher() as buscador:
        resultados = buscador.search(consulta)
        for resultado in resultados:
            if resultado["seccion"] == seccion:
                return True
    return False

# 7. Ejecución

In [19]:
contenido = leer_fichero("./ejemplo1.txt")
secciones = extraer_secciones(contenido)
sustantivos = {}

for seccion, texto in secciones.items():
    texto_limpio = limpiar_texto(texto)
    sustantivos[seccion] = extraer_sustantivos(texto_limpio)

# Imprimir resultado
for seccion, palabras in sustantivos.items():
    print(f"\nSustantivos en {seccion}: {', '.join(palabras) if palabras else 'No hay sustantivos'}")


Sustantivos en titulo: día, muertes

Sustantivos en noticia: virus, final, pandemia, fase, balance, daños, expertos, tiempo, registros, semanas, España

Sustantivos en resumen: capacidad, serie, hospitalizados, registro, personas, infección


# 8. Buscador de sustantivos

In [23]:
# Crear el índice de Whoosh
crear_indice(sustantivos)

# Interacción con el usuario
seccion_usuario = input("En qué sección quieres buscar (titulo, noticia, resumen)?: ").strip().lower()
sustantivo_usuario = input("Qué sustantivo quieres encontrar?: ").strip().lower()

if buscar_sustantivo(seccion_usuario, sustantivo_usuario):
    print(f"El sustantivo '{sustantivo_usuario}' SÍ está en la sección '{seccion_usuario}'.")
else:
    print(f"El sustantivo '{sustantivo_usuario}' NO está en la sección '{seccion_usuario}'.")

El sustantivo 'día' SÍ está en la sección 'titulo'.
